In [1]:
import pandas as pd

import dms_variants
import dms_variants.codonvarianttable
from dms_variants.constants import CBPALETTE
import dms_variants.plotnine_themes

from plotnine import *

_ = theme_set(dms_variants.plotnine_themes.theme_graygrid())


In [2]:
import os
os.chdir('../')

In [31]:
bc_runs

,date,virus_batch,library,sample_type,antibody,antibody_concentration,replicate,fastq_R1,exclude_after_counts,notes,sample,library_sample
11,221027,1,libA,antibody,3x-1C04_5G04,22.000000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,yes,NaN,221027_1_antibody_3x-1C04_5G04_22.0_1,libA_221027_1_antibody_3x-1C04_5G04_22.0_1
12,221027,1,libA,antibody,3x-1C04_5G04,14.670000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_14.67_1,libA_221027_1_antibody_3x-1C04_5G04_14.67_1
13,221027,1,libA,antibody,3x-1C04_5G04,9.780000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_9.78_1,libA_221027_1_antibody_3x-1C04_5G04_9.78_1
14,221027,1,libA,antibody,3x-1C04_5G04,6.520000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_6.52_1,libA_221027_1_antibody_3x-1C04_5G04_6.52_1
15,221027,1,libA,antibody,3x-1C04_5G04,4.350000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_4.35_1,libA_221027_1_antibody_3x-1C04_5G04_4.35_1
16,221027,1,libA,antibody,3x-1C04_5G04,2.900000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_2.9_1,libA_221027_1_antibody_3x-1C04_5G04_2.9_1
17,221027,1,libA,antibody,3x-1C04_5G04,1.930000,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_3x-1C04_5G04_1.93_1,libA_221027_1_antibody_3x-1C04_5G04_1.93_1
18,221027,1,libA,antibody,AUSAB-13_2-fold,0.015040,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_AUSAB-13_2-fold_0.01504_1,libA_221027_1_antibody_AUSAB-13_2-fold_0.01504_1
19,221027,1,libA,antibody,AUSAB-13_2-fold,0.007520,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_AUSAB-13_2-fold_0.00752_1,libA_221027_1_antibody_AUSAB-13_2-fold_0.00752_1
20,221027,1,libA,antibody,AUSAB-13_2-fold,0.003760,1,['/fh/fast/bloom_j/SR/ngs/illumina/fwelsh/2210...,no,NaN,221027_1_antibody_AUSAB-13_2-fold_0.00376_1,libA_221027_1_antibody_AUSAB-13_2-fold_0.00376_1


In [32]:
bc_runs = pd.read_csv('results/barcode_runs/processed_barcode_runs.csv')
bc_runs = bc_runs.loc[bc_runs['date'] != 221021]
samples = bc_runs['library_sample'].to_list()
libs = bc_runs['library'].to_list()
antibody = bc_runs['antibody'].to_list()
antibody_concentration = bc_runs['antibody_concentration'].to_list()

valid_bc_counts = []

for sample in samples:
    fates = pd.read_csv(f'results/barcode_runs/fates_by_sample/{sample}.csv')
    valid_bc = fates.query('fate == "valid barcode"')['count'].item()
    valid_bc_counts.append(valid_bc)

In [35]:
bc_counts_df = pd.DataFrame(
    {'library': libs,
     'sample': samples,
     'antibody': antibody,
     'antibody_concentration': antibody_concentration,
     'valid_bc_counts': valid_bc_counts       
    }
)

bc_counts_df.head()

,library,sample,antibody,antibody_concentration,valid_bc_counts
0,libA,libA_221027_1_antibody_3x-1C04_5G04_22.0_1,3x-1C04_5G04,22.00,221571
1,libA,libA_221027_1_antibody_3x-1C04_5G04_14.67_1,3x-1C04_5G04,14.67,5231491
2,libA,libA_221027_1_antibody_3x-1C04_5G04_9.78_1,3x-1C04_5G04,9.78,4923635
3,libA,libA_221027_1_antibody_3x-1C04_5G04_6.52_1,3x-1C04_5G04,6.52,3995874
4,libA,libA_221027_1_antibody_3x-1C04_5G04_4.35_1,3x-1C04_5G04,4.35,4599237


In [48]:
p = (
    ggplot(
        bc_counts_df,
        aes(x='antibody_concentration', y='valid_bc_counts', fill='library') + 
    geom_bar(stat='identity', position=position_dodge()) +
    theme(figure_size=(10, 5),
         panel_grid_major_x=element_blank(),
         ) +
    scale_fill_manual(values=CBPALETTE[1:]))
)
#     labs(x='prob_escape', 
#          y='# variants w/ any mutation at 1C04 mAb targeted site',
#          title='Prob escape of variants with 1C04 mutation at diff 1C04:5G04 cocktail concentrations, min prob escape = 0.1'
#         )


p = p + facet_wrap('~ antibody')

# p = p + facet_wrap('~ antibody_concentration', nrow = 6)
_ = p.draw()

# _ = p.draw()

# p.save('scratch_notebooks/221031_modeling-selections-test/high_prob_escape_1C04_var.pdf')

PlotnineError: "Cannot add layer to object of type <class 'plotnine.mapping.aes.aes'>"